In [109]:
import pandas as pd
import os
import duckdb

In [110]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/workspaces/spotify_insights/airflow/credentials/google_credentials.json'
airport_location_path = "gs://spotify-insights-source-data/airport_code_location/airports-code.csv"
user_location_path = "gs://spotify-insights-source-data/user_location/joseph.higaki_user_location.csv"
streams_source_path = "gs://spotify-insights-pipeline-data/stg_spotify_streams/dtype_dedup"

In [9]:


airport_location = pd.read_csv(airport_location_path, delimiter=";")
airport_location.rename(columns={'Airport Code': 'airport_code'}, inplace=True)
airport_location


,airport_code,Airport Name,City Name,Country Name,Country Code,Latitude,Longitude,World Area Code,City Name geo_name_id,Country Name geo_name_id,coordinates
0,CRS,Corsicana,Corsicana,United States,US,32.100000,-96.466667,67,4683462,6252001.0,"32.1, -96.466667"
1,ACP,Sahand,Sahand,Iran,IR,37.733333,46.450000,632,\N,130758.0,"37.733333, 46.45"
2,TBE,Timbunke,Timbunke,Papua New Guinea,PG,-4.183333,143.516667,804,\N,2088628.0,"-4.183333, 143.516667"
3,LWO,Snilow,Lviv,Ukraine,UA,49.812500,23.956111,488,702550,690791.0,"49.8125, 23.956111"
4,SCP,St Crepin,St Crepin,France,FR,44.683333,6.583333,427,\N,3017382.0,"44.683333, 6.583333"
...,...,...,...,...,...,...,...,...,...,...,...
9182,SJD,Los Cabos,San Jose Del Cabo,Mexico,MX,23.151944,-109.721111,148,\N,3996063.0,"23.151944, -109.721111"
9183,HVS,Municipal,Hartsville,United States,US,34.400000,-80.116667,67,4628109,6252001.0,"34.4, -80.116667"
9184,GDG,Magdagachi,Magdagachi,Russian Federation,RU,53.466667,125.800000,770,2020591,2017370.0,"53.466667, 125.8"
9185,CHO,Albemarle,Charlottesville,United States,US,38.133333,-78.450000,67,4752031,6252001.0,"38.133333, -78.45"


In [10]:
user_location = pd.read_csv(user_location_path)

user_location['start_at'] = user_location['start_at'].astype("datetime64[ns, UTC]") 
user_location['end_at'] = user_location['end_at'].astype("datetime64[ns, UTC]") 
user_location

,username,start_at,end_at,airport_code
0,joseph.higaki,2014-01-01 00:00:00+00:00,2020-10-17 04:59:59+00:00,LIM
1,joseph.higaki,2020-10-17 05:00:00+00:00,2024-09-10 04:59:59+00:00,BCN
2,joseph.higaki,2024-06-20 18:35:00+00:00,2024-06-24 09:59:59+00:00,CDG
3,joseph.higaki,2024-06-24 10:00:00+00:00,2024-06-25 00:15:59+00:00,LHR
4,joseph.higaki,2020-06-25 00:16:00+00:00,2024-09-10 04:59:59+00:00,BCN
5,joseph.higaki,2024-09-10 05:00:00+00:00,2024-09-17 04:59:59+00:00,BIQ
6,joseph.higaki,2024-09-17 05:00:00+00:00,2024-12-19 04:59:59+00:00,BCN
7,alice.alziati,2010-01-01 00:00:00+00:00,2024-11-17 04:59:59+00:00,BCN
8,alice.alziati,2024-11-17 05:00:00+00:00,2024-11-20 04:59:59+00:00,VCE
9,alice.alziati,2024-11-20 05:00:00+00:00,2024-12-20 04:59:59+00:00,BCN


In [11]:
con =  duckdb.connect()
con.register('airport_location', airport_location)
con.register('user_location', user_location)


# Extract List to Download weather from
Generate the list of years from the users location history 

In [12]:
query = """
    with recursive user_location_min_max as (
        select 
            extract('year' FROM min(start_at))  as min_year,
            extract('year' FROM max(end_at))  as max_year
        from user_location
    ),
    years_list as (
        select min_year as year
        from user_location_min_max
        union all 
        select year + 1
        from years_list
        join user_location_min_max on year < max_year
    )      
    select y.year, u.username, u.airport_code
    from years_list y
    join user_location u 
        on y.year between 
            extract('year' FROM u.start_at) 
            and extract('year' FROM u.end_at) 
    order by username, year
    """        
df = con.execute(query).fetchdf()
df

,year,username,airport_code
0,2010,alice.alziati,BCN
1,2011,alice.alziati,BCN
2,2012,alice.alziati,BCN
3,2013,alice.alziati,BCN
4,2014,alice.alziati,BCN
5,2015,alice.alziati,BCN
6,2016,alice.alziati,BCN
7,2017,alice.alziati,BCN
8,2018,alice.alziati,BCN
9,2019,alice.alziati,BCN


## Years and coordinates from users

In [16]:
query = """
    with recursive user_location_min_max as (
        select 
            extract('year' FROM min(start_at))  as min_year,
            extract('year' FROM max(end_at))  as max_year
        from user_location
    ),
    years_list as (
        select min_year as year
        from user_location_min_max
        union all 
        select year + 1
        from years_list
        join user_location_min_max on year < max_year
    ),
    years_code as (
        select distinct y.year, u.airport_code
        from years_list y
        join user_location u 
            on y.year between 
                extract('year' FROM u.start_at) 
                and extract('year' FROM u.end_at)         
    ),
    years_coordinates as (
        select yc.year, yc.airport_code, a.Latitude, a.Longitude
        from years_code yc 
        join airport_location a on a.airport_code = yc.airport_code
    )
    select year, airport_code, Latitude as latitude, Longitude as longitude
    from years_coordinates
    order by year, Latitude
    """        
years_coordinates = con.execute(query).fetchdf()
years_coordinates

,year,airport_code,latitude,longitude
0,2010,BCN,41.300000,2.083333
1,2011,BCN,41.300000,2.083333
2,2012,BCN,41.300000,2.083333
3,2013,BCN,41.300000,2.083333
4,2014,LIM,-12.021944,-77.114444
5,2014,BCN,41.300000,2.083333
6,2015,LIM,-12.021944,-77.114444
7,2015,BCN,41.300000,2.083333
8,2016,LIM,-12.021944,-77.114444
9,2016,BCN,41.300000,2.083333


In [64]:
type(years_coordinates)

pandas.core.frame.DataFrame

In [2]:
year = '2024'
latitude = 41.300000
longitude = 2.083333

In [3]:
import requests
import pandas as pd


# Define the API URL
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 41.300000,
    "longitude": 2.083333,
    "start_date": "2024-01-01",
    "end_date": "2024-01-02",
    "hourly": "temperature_2m,cloud_cover",
    "daily": "weather_code,sunrise,sunset,daylight_duration,sunshine_duration",
}

# Make the GET request
response = requests.get(url, params=params)
response.raise_for_status()  # Raise an error if the request fails

# Parse the JSON response
data = response.json()

# Extract daily data
daily_data = data["daily"]

# Create a DataFrame
daily_df = pd.DataFrame(daily_data)

# Convert the "time" column to datetime with UTC timezone
daily_df["time"] = pd.to_datetime(daily_df["time"], format="%Y-%m-%d").dt.tz_localize("UTC")

# Convert numerical columns to float
numerical_columns = ["weather_code", "daylight_duration", "sunshine_duration"]
daily_df[numerical_columns] = daily_df[numerical_columns].astype(float)

# Rename columns for clarity
daily_df.rename(columns={"time": "date"}, inplace=True)

# Print the resulting DataFrame



In [7]:
daily_df

,date,weather_code,sunrise,sunset,daylight_duration,sunshine_duration
0,2024-01-01 00:00:00+00:00,3.0,2024-01-01T07:17,2024-01-01T16:32,33306.56,20614.69
1,2024-01-02 00:00:00+00:00,51.0,2024-01-02T07:17,2024-01-02T16:33,33351.89,18128.98


In [6]:
# Extract hourly data
hourly_data = data["hourly"]

# Create a DataFrame
hourly_df = pd.DataFrame(hourly_data)

# Convert the "time" column to datetime with UTC timezone
hourly_df["time"] = pd.to_datetime(hourly_df["time"], format="%Y-%m-%dT%H:%M").dt.tz_localize("UTC")

# Convert numerical columns to float
numerical_columns = ["temperature_2m", "cloud_cover"]
hourly_df[numerical_columns] = hourly_df[numerical_columns].astype(float)

# Rename columns for clarity
hourly_df.rename(columns={"time": "date"}, inplace=True)

# Print the resulting DataFrame
print(hourly_df)


                        date  temperature_2m  cloud_cover
0  2024-01-01 00:00:00+00:00             7.3        100.0
1  2024-01-01 01:00:00+00:00             7.4        100.0
2  2024-01-01 02:00:00+00:00             7.6        100.0
3  2024-01-01 03:00:00+00:00             7.0        100.0
4  2024-01-01 04:00:00+00:00             7.1        100.0
5  2024-01-01 05:00:00+00:00             6.0        100.0
6  2024-01-01 06:00:00+00:00             6.4        100.0
7  2024-01-01 07:00:00+00:00             7.0        100.0
8  2024-01-01 08:00:00+00:00             7.3        100.0
9  2024-01-01 09:00:00+00:00             8.5        100.0
10 2024-01-01 10:00:00+00:00            10.8        100.0
11 2024-01-01 11:00:00+00:00            11.8        100.0
12 2024-01-01 12:00:00+00:00            12.8         71.0
13 2024-01-01 13:00:00+00:00            13.8         14.0
14 2024-01-01 14:00:00+00:00            13.9         37.0
15 2024-01-01 15:00:00+00:00            13.6          0.0
16 2024-01-01 

# Retrieving Information for year / coordinates   

In [4]:
source_path = "gs://spotify-insights-pipeline-data/weather_location/frequency=daily/"

In [5]:
daily_data = pd.read_parquet(source_path, engine="pyarrow")
daily_data

,date,weather_code,sunrise,sunset,daylight_duration,sunshine_duration,year,latitude,longitude
0,2023-01-01 00:00:00+00:00,3.0,2023-01-01T07:17,2023-01-01T16:32,33310.58,25135.09,2023,41.3,2.083
1,2023-01-02 00:00:00+00:00,3.0,2023-01-02T07:17,2023-01-02T16:33,33356.70,12431.39,2023,41.3,2.083
2,2023-01-03 00:00:00+00:00,51.0,2023-01-03T07:17,2023-01-03T16:34,33406.95,23423.65,2023,41.3,2.083
3,2023-01-04 00:00:00+00:00,2.0,2023-01-04T07:17,2023-01-04T16:35,33461.20,28857.10,2023,41.3,2.083
4,2023-01-05 00:00:00+00:00,0.0,2023-01-05T07:17,2023-01-05T16:36,33519.34,29073.38,2023,41.3,2.083
...,...,...,...,...,...,...,...,...,...
360,2023-12-27 00:00:00+00:00,0.0,2023-12-27T07:16,2023-12-27T16:28,33139.60,28992.14,2023,41.3,2.083
361,2023-12-28 00:00:00+00:00,3.0,2023-12-28T07:16,2023-12-28T16:29,33164.76,12936.24,2023,41.3,2.083
362,2023-12-29 00:00:00+00:00,3.0,2023-12-29T07:16,2023-12-29T16:30,33194.11,14400.00,2023,41.3,2.083
363,2023-12-30 00:00:00+00:00,3.0,2023-12-30T07:17,2023-12-30T16:30,33227.59,28925.73,2023,41.3,2.083


In [6]:

hourly_data = pd.read_parquet("gs://spotify-insights-pipeline-data/weather_location/frequency=hourly/", engine="pyarrow")
hourly_data

,datetime,temperature_2m,cloud_cover,year,latitude,longitude
0,2023-01-01 00:00:00+00:00,11.6,100.0,2023,41.3,2.083
1,2023-01-01 01:00:00+00:00,10.6,100.0,2023,41.3,2.083
2,2023-01-01 02:00:00+00:00,9.1,99.0,2023,41.3,2.083
3,2023-01-01 03:00:00+00:00,8.2,100.0,2023,41.3,2.083
4,2023-01-01 04:00:00+00:00,8.0,100.0,2023,41.3,2.083
...,...,...,...,...,...,...
8755,2023-12-31 19:00:00+00:00,10.0,79.0,2023,41.3,2.083
8756,2023-12-31 20:00:00+00:00,9.0,100.0,2023,41.3,2.083
8757,2023-12-31 21:00:00+00:00,8.4,86.0,2023,41.3,2.083
8758,2023-12-31 22:00:00+00:00,7.9,100.0,2023,41.3,2.083


## Only fetch the MISSING years / coordinates from the weather cache data

`years_coordinates`  - `weather-info`.(year, lat, long) 

In [37]:
def validate_frequency(frequency: str):
    if frequency not in ["daily", "hourly"]:
        raise ValueError(f"Invalid frequency: {frequency}")

In [50]:
#years_coordinates


def get_weather_cached_data(frequency, year, latitude, longitude):
    validate_frequency(frequency)
    year = year 
    latitude = round(float(latitude), 3) 
    longitude = round(float(longitude), 3) 
    print(year, latitude, longitude)
    data = None
    try:
        path = f"gs://spotify-insights-pipeline-data/weather_location/frequency={frequency}/year={year}/latitude={latitude}/longitude={longitude}"
        print(path)
        data = pd.read_parquet(path, engine="pyarrow")
    except FileNotFoundError:
        print(f"Data not found for year={year}, latitude={latitude}, longitude={longitude}")    
    return data

In [52]:
get_weather_cached_data('daily', 2022, 41.300, 2.083)

2022 41.3 2.083
gs://spotify-insights-pipeline-data/weather_location/frequency=daily/year=2022/latitude=41.3/longitude=2.083
Data not found for year=2022, latitude=41.3, longitude=2.083


In [57]:
def exists_weather_cached_data(frequency, year, latitude, longitude):
    data = get_weather_cached_data(frequency, year, latitude, longitude)
    if frequency == 'daily':
        return data is not None
    elif frequency == 'hourly':
        return data is not None

In [59]:
exists_weather_cached_data('daily', 2023, 41.300, 2.083)    

2023 41.3 2.083
gs://spotify-insights-pipeline-data/weather_location/frequency=daily/year=2023/latitude=41.3/longitude=2.083


True

In [62]:
years_coordinates

,year,airport_code,latitude,longitude
0,2010,BCN,41.300000,2.083333
1,2011,BCN,41.300000,2.083333
2,2012,BCN,41.300000,2.083333
3,2013,BCN,41.300000,2.083333
4,2014,LIM,-12.021944,-77.114444
5,2014,BCN,41.300000,2.083333
6,2015,LIM,-12.021944,-77.114444
7,2015,BCN,41.300000,2.083333
8,2016,LIM,-12.021944,-77.114444
9,2016,BCN,41.300000,2.083333


In [108]:

hourly_data = pd.read_parquet("gs://spotify-insights-pipeline-data/weather_location/frequency=hourly/", engine="pyarrow")
hourly_data


,datetime,temperature_2m,cloud_cover,year,latitude,longitude
0,2010-01-01 00:00:00+00:00,11.3,5.0,2010,41.3,2.083
1,2010-01-01 01:00:00+00:00,10.6,5.0,2010,41.3,2.083
2,2010-01-01 02:00:00+00:00,9.9,32.0,2010,41.3,2.083
3,2010-01-01 03:00:00+00:00,9.3,91.0,2010,41.3,2.083
4,2010-01-01 04:00:00+00:00,8.9,88.0,2010,41.3,2.083
...,...,...,...,...,...,...
227875,2024-12-30 19:00:00+00:00,NaN,NaN,2024,51.47,-0.451
227876,2024-12-30 20:00:00+00:00,NaN,NaN,2024,51.47,-0.451
227877,2024-12-30 21:00:00+00:00,NaN,NaN,2024,51.47,-0.451
227878,2024-12-30 22:00:00+00:00,NaN,NaN,2024,51.47,-0.451


In [111]:

daily_data = pd.read_parquet("gs://spotify-insights-pipeline-data/weather_location/frequency=daily/", engine="pyarrow")
daily_data

,date,weather_code,sunrise,sunset,daylight_duration,sunshine_duration,year,latitude,longitude
0,2010-01-01 00:00:00+00:00,3.0,2010-01-01T07:17,2010-01-01T16:33,33324.00,28849.25,2010,41.3,2.083
1,2010-01-02 00:00:00+00:00,3.0,2010-01-02T07:17,2010-01-02T16:33,33369.09,28971.10,2010,41.3,2.083
2,2010-01-03 00:00:00+00:00,3.0,2010-01-03T07:17,2010-01-03T16:34,33417.85,12335.88,2010,41.3,2.083
3,2010-01-04 00:00:00+00:00,61.0,2010-01-04T07:17,2010-01-04T16:35,33470.23,9917.08,2010,41.3,2.083
4,2010-01-05 00:00:00+00:00,55.0,2010-01-05T07:17,2010-01-05T16:36,33526.18,3128.02,2010,41.3,2.083
...,...,...,...,...,...,...,...,...,...
9490,2024-12-26 00:00:00+00:00,3.0,2024-12-26T08:07,2024-12-26T15:58,28274.55,888.77,2024,51.47,-0.451
9491,2024-12-27 00:00:00+00:00,3.0,2024-12-27T08:07,2024-12-27T15:59,28311.84,0.00,2024,51.47,-0.451
9492,2024-12-28 00:00:00+00:00,3.0,2024-12-28T08:07,2024-12-28T15:59,28355.26,0.00,2024,51.47,-0.451
9493,2024-12-29 00:00:00+00:00,3.0,2024-12-29T08:07,2024-12-29T16:00,28404.74,NaN,2024,51.47,-0.451
